In [1]:
class Flight:
    def __init__(self, start, dest, cost, start_time, landing_time, capacity):
        self.start = start
        self.dest = dest
        self.cost = cost
        self.start_time = start_time
        self.landing_time = landing_time
        self.capacity = capacity

In [2]:
def get_node_info_from_index(node_index, sg, nsg):
    """ For a given qubit index, returns the corresponding time slice and the node label. """

    for time_slice in range(len(nsg)):
        if node_index in nsg[time_slice]:

            pos = nsg[time_slice].index(node_index)
            node_label = sg[time_slice][pos]

            return time_slice, node_label

In [3]:
def result_evaluation(result, offset, sg, nsg, cost_matrix):
    """
    For a given qubit string, returns an explanation of the nodes that are passed per time slice.
    """
    counter = 0
    previous_node = None
    total_cost = 0
    solution_not_valid = False

    for i in range(len(result)):
        if result[i] == '1':
            time_slice, node_label = get_node_info_from_index(i + offset, sg, nsg)
            if time_slice != counter:
                solution_not_valid = True

            if counter == 0:
                previous_node = node_label
            else:
                cost = cost_matrix[previous_node][node_label]
                total_cost += cost
                print(('\t\t\t\tcost: {}').format(cost))
                previous_node = node_label


            print(('node {} at time slice {}').format(node_label, time_slice))

            counter += 1

            
    print(('\ntotal cost: {}').format(total_cost))
    if total_cost >= 500 or solution_not_valid:
        print('\nIt seems like no valid solution could be found. This solution disregards at least one constraint.')

    return

In [4]:
def get_unique_nodes(edge_list):
    """
    Creates a list of all unique nodes appearing in a given graph.
    The start node is added by default; every other node is only included
    if it can be reached from another node. Isolated nodes will not be 
    added to the list, unless they happen to be the start node.
    """
    
    nr_nodes = 0
    unique_nodes = []

    for edge in edge_list:
        if edge[0] not in unique_nodes:
            unique_nodes.append(edge[0])
        if edge[1] not in unique_nodes:
            unique_nodes.append(edge[1])

    return unique_nodes

In [5]:
def construct_sliced_graph(flight_dict, start, dest):
    """
    Constructs a two-dimensional array from a list of edges. Each subarray represents 
    one time slice. The returned array contains the specified start node in the 
    first sub-array; based on this start node, every node that can be reached in 
    n steps is present in the nth subarray. Nodes that can be reached in a different 
    amount of steps are present in every fitting subarray.
    """

    # we define a new dict 'flights' here, this is local
    # gets updated with the zero-cost edge with both start and destination = dest
    flight_dict.update({'Z' + str(dest) : Flight(dest, dest, 0,0,0,0)})
    flights = flight_dict.copy()
    
    for f in list(flights.keys()):
        if f[0] == 'Z' and f[1:] != str(dest):
            del flights[f]


    sg = []
    sg.append([flight for flight in flights if flights[flight].start == start])

    nr_unique_nodes = len(get_unique_nodes(flights))

    for k in range(nr_unique_nodes):
        if len(sg[k]) != 0 and not (len(sg[k]) == 1 and flights[sg[k][0]].dest == dest):
            sg.append([])
            for f1 in flights:
                for f2 in sg[k]:
                    if flights[f1].start == flights[f2].dest:
                        sg[k+1].append(f1)
                        break 
        else:
            break

    # sort out any flights in the last slice that do not lead to destination
    sg[-1] = [f for f in sg[-1] if flights[f].dest == dest]

    # going back, we can eliminate edges (flights) that won't lead 
    # to places that the flights in the next slice depart from, 
    # a.k.a. dead ends

    for i in range(len(sg) - 1):
        edges_to_remove = []
        index = len(sg) - 1 - i  # since we count backwards

        for j in sg[index - 1]:
            leads_to_next_slice = 0  # number of edges in the next slice that sg[index-1][j] is connected to
            dest_j = flights[j].dest
            for k in sg[index]:

                # for each node in the next slice, add 1 if sg[index-1][j] is connected to it, 0 if not
                if dest_j == flights[k].start:
                    leads_to_next_slice += 1
                

            # if leads_to_next_slice is zero, it means sg[index-1][j] is not connected to any node in the next slice; essentially a dead end. 
            # it is removed, and any node in the previous slice that only connected to it will thus also be a dead end, and will be removed. etc., etc.
            if leads_to_next_slice == 0:
                edges_to_remove.append(j)

        for edge in edges_to_remove:
            sg[index-1].remove(edge)

    slices_to_remove = []
    for i in range(len(sg)):
        index = len(sg) - 1 - i

        if len(sg[index]) == 1 and flights[sg[index][0]].dest == dest:
            slices_to_remove.append(index)
        else:
            break
    
    if len(slices_to_remove) > 1:
        for i in slices_to_remove[:-1]:
            del sg[i]

    return sg

In [6]:
def construct_numbered_sliced_graph(sg, offset):
    """
    Based on a two-dimensional array that constructed time slices from a graph,
    renumbers the contents by giving each element its index in the corresponding 
    flattened array. Since the numbered sliced graph is used for getting the qubit 
    indices of the nodes, and since there are several sliced graphs, an offset is
    necessary.
    Example: With input ([[0], [1, 3], [2, 3]], 0), output would be [[0], [1, 2], [3, 4]].
    With input ([[0], [1, 3], [2, 3], [4, 3]], 5), output would be [[5], [6, 7], [8, 9], [10, 11]].
    """
    
    nsg = [] # numbered sliced graph
    node_counter = 0

    for m in range(len(sg)):
        nsg.append([])
        for n in range(len(sg[m])):
            nsg[m].append(node_counter + offset)
            node_counter+=1
    
    return nsg, node_counter

In [7]:
def flight_cost(flight):
    """
    Returns the flight cost for a given flight, identified by its flight number.
    """
    return flights[flight].cost

In [8]:
def transfer_check(flight_1, flight_2, passenger_index):
    """
    TO DO

    For two given flights, checks if the time inbetween is 
    smaller than the minimum transfer time. If yes, returns 
    the specified penalty value. If no, returns 0.
    """

    dest_flight_1 = flights[flight_1].dest
    start_flight_2 = flights[flight_2].start

    destination = passengers[passenger_index]

    if dest_flight_1 == start_flight_2 and dest_flight_1 == destination:
        return 0
    elif dest_flight_1 != start_flight_2: # the flights are not connecting flights
        return penalty
    else:
        landing_time = flights[flight_1].landing_time
        start_time = flights[flight_2].start_time

        if start_time - landing_time < 1: #Test_min_transfer_in_min:
            return penalty
        else:
            return 0

def transfer_time(flight_1, flight_2):
    """For two given flights, returns the time transfer time."""

    landing_time = flights.get(flight_1)[4]
    start_time = flights.get(flight_2)[3]

    transfer = start_time - landing_time

    if transfer < 0:
        return 0
    else:
        return transfer

In [9]:
def connection_check(flight_1, flight_2):
    """
    For two given flights, checks if the destination of
    the first is the start of the second, i.e. if they are
    connecting flights. If yes, return 0, if not, return
    the specified penalty value.
    """

    destination = flights[flight_1].landing_time #[1] #.landing_time
    start = flights[flight_2].start_time #[0]

    if destination != start:
        return penalty
    else:
        return 0

In [10]:
def capacity_polynomial(flight, nrpassengers):

    if flight[0] == 'Z':
        return 0
    
    qubit_indices = []

    for k in range(len(passengers)):
        for c in range(len(test_sg[k])):
            for e in range(len(test_sg[k][c])):
                if test_sg[k][c][e] == flight:
                    qubit_indices.append(X[test_nsg[k][c][e]])
    
    capacity = flights[flight].capacity #[5]

    sum_vars = 0
    for i in qubit_indices:
        sum_vars += i

    #return sum_vars
    global nr_qubits

    #nr_addends = nrpassengers if nrpassengers < capacity else capacity # maybe one could just reference the passenger var
    #nr_addends = nr_addends if nr_addends < len(qubit_indices) else len(qubit_indices)
    nr_addends = capacity

    #print(nr_addends)

    if nr_addends == 0:
        return 0  # this is okay if all flights with capacity 0 are sorted out at the beginning
    #elif nr_addends == 1:
    #    return P * (sum_vars - 1) * sum_vars
    else:
        # we need nr_addends - 1 vars
        polynomial = 0
        minuends = [0]

        for i in qubit_indices:
            minuends[0] += i 

        for i in range(1, nr_addends):
            minuends.append(0)
            for j in range(i, nr_addends):
                minuends[i] += X[nr_qubits + j - 1]
        minuends.append(0)

        print(minuends)
            
        for i in range(len(minuends) - 1):
            polynomial += P * ((minuends[i] - (nr_addends - i)) * (minuends[i] - minuends[i + 1]))

            #polynomial += -20 * P * ((  ((minuends[i])**2)**(1/2) - ((nr_addends - i)**2)**(1/2)  ) * (   ((minuends[i])**2)**(1/2) - ((minuends[i + 1])**2)**(1/2)   ))



            #polynomial += P * (( (nr_addends - i)  -  minuends[i] ) * ( minuends[i] - minuends[i + 1]))


        nr_qubits += nr_addends - 1

        return polynomial

In [11]:
import sympy as sym
import matplotlib.pyplot as plt
import pandas as pd
import time

In [12]:
# data
flights = {
    'A11': Flight(0, 1, 150, 8, 9, 3),
    'A12': Flight(0, 2, 20, 8, 11, 3),
    'A13': Flight(2, 1, 70, 11, 12, 3),
    'A14': Flight(1, 3, 30, 11, 12, 1),
    'A15': Flight(2, 3, 200, 11, 12, 1)
}

start_node = 0
passengers = [1, 1, 1]

penalty = 5000

test_sg = [
    [
        ['A11', 'A12'],
        ['A13', 'A14', 'A15', 'B00'],
        ['A14', 'B00']
    ],
    [
        ['A11', 'A12'],
        ['A13', 'A14', 'A15'],
        ['A14', 'C00']
    ]
]

In [13]:
test_sg = []
test_nsg = []
nodes_total = 0

for k in range(len(passengers)):
    test_sg.append(construct_sliced_graph(flights, start_node, passengers[k]))

    numbered_subgraph, nr_nodes_subgraph = construct_numbered_sliced_graph(test_sg[k], offset=nodes_total)
    test_nsg.append(numbered_subgraph)

    nodes_total += nr_nodes_subgraph

nr_qubits = nodes_total
nr_qubits

15

In [14]:
test_sg

[[['A11', 'A12'], ['A13', 'Z1'], ['Z1']],
 [['A11', 'A12'], ['A13', 'Z1'], ['Z1']],
 [['A11', 'A12'], ['A13', 'Z1'], ['Z1']]]

In [15]:
test_nsg

[[[0, 1], [2, 3], [4]], [[5, 6], [7, 8], [9]], [[10, 11], [12, 13], [14]]]

test_nsg = [
    [
        [0, 1],
        [2, 3, 4, 5],
        [6, 7]
    ],
    [
        [8, 9],
        [10, 11, 12],
        [13, 14]
    ]
]

In [16]:
flight_list = list(flights.keys())
flight_list

['A11', 'A12', 'A13', 'A14', 'A15', 'Z1']

In [17]:
k = sym.symbols('k') # passengers
c = sym.symbols('c') # slices
e = sym.symbols('e') # edges
f = sym.symbols('f') # edges
X = sym.IndexedBase('X') # binary vars for qubits
P = sym.symbols('P') # penalty

edge1 = sym.symbols('edge1') # edges
edge2 = sym.symbols('edge2') # edges

nrpassengers = sym.symbols('nrpassengers')
nrflights = sym.symbols('nrflights')

nrslices = sym.IndexedBase('nrslices')
nredges = sym.IndexedBase('nredges')

sg = sym.IndexedBase('sg')
nsg = sym.IndexedBase('nsg')

validtransfer = sym.IndexedBase('validtransfer')
transfertime = sym.IndexedBase('transfertime')
connection = sym.IndexedBase('connection')
cost = sym.IndexedBase('cost')
flightduration = sym.IndexedBase('flightduration')
capacity = sym.IndexedBase('capacity')

flightlist = sym.IndexedBase('flightlist')

In [18]:
# only one flight in each time slice
cost_function = sym.Sum(
    sym.Sum(
        P * (sym.Sum(
            X[nsg[k,c,e]]
            , (e, 0, nredges[k,c])
    ) - 1)**2
        , (c, 0, nrslices[k] - 1)
    )
    , (k, 0, nrpassengers-1)
)
cost_function

Sum(P*(Sum(X[nsg[k, c, e]], (e, 0, nredges[k, c])) - 1)**2, (c, 0, nrslices[k] - 1), (k, 0, nrpassengers - 1))

In [19]:
# only one flight in each time slice
# cost of flights
cost_function = sym.Sum(
    sym.Sum(
        P * (sym.Sum(
            X[nsg[k,c,e]]
            , (e, 0, nredges[k,c])
    ) - 1)**2 + 
    sym.Sum(
        X[nsg[k,c,e]], (e, 0, nredges[k,c])
    )
        , (c, 0, nrslices[k] - 1)
    )
    , (k, 0, nrpassengers-1)
)
cost_function

Sum(P*(Sum(X[nsg[k, c, e]], (e, 0, nredges[k, c])) - 1)**2 + Sum(X[nsg[k, c, e]], (e, 0, nredges[k, c])), (c, 0, nrslices[k] - 1), (k, 0, nrpassengers - 1))

In [20]:
# only one flight in each time slice
# cost of flights
# enough transfer time
cost_function = sym.Sum(
    sym.Sum(
        P * (sym.Sum(
            X[nsg[k,c,e]]
            , (e, 0, nredges[k,c]-1)
    ) - 1)**2
    # + 
    #sym.Sum(
    #    X[nsg[k,c,e]], (e, 0, nredges[k,c]-1)
    #)
        , (c, 0, nrslices[k]-1)
    ) + 2 *
    sym.Sum(
        0.5 *
        sym.Sum(
            X[nsg[k,c,e]] * cost[sg[k,c,e]], (e, 0, nredges[k,c]-1)
    )
        , (c, 0, nrslices[k]-1)
    ) + 

    sym.Sum( 2 *
        sym.Sum( 0.5 *
            sym.Sum(
                (X[nsg[k,c,edge1]] * X[nsg[k,c+1,edge2]] * validtransfer[sg[k,c,edge1], sg[k,c+1,edge2], k])                
                , (edge2, 0, nredges[k,c+1]-1)
            )
            , (edge1, 0, nredges[k,c]-1)
        )
        , (c, 0, nrslices[k]-2)
    )
    
    
    #+ 
    #sym.Sum( 2 *
    #    sym.Sum( 0.5 *
    #        sym.Sum(
    #            
    #            (X[nsg[k,c,edge1]] * X[nsg[k,c+1,edge2]] * connection[sg[k,c,edge1], sg[k,c+1,edge2]])
    #            , (edge2, 0, nredges[k,c+1]-1)
    #        )
    #        , (edge1, 0, nredges[k,c]-1)
    #    )
    #    , (c, 0, nrslices[k]-2)
    #)
    , (k, 0, nrpassengers-1)
)
cost_function

Sum(Sum(P*(Sum(X[nsg[k, c, e]], (e, 0, nredges[k, c] - 1)) - 1)**2, (c, 0, nrslices[k] - 1)) + 2*Sum(0.5*Sum(X[nsg[k, c, e]]*cost[sg[k, c, e]], (e, 0, nredges[k, c] - 1)), (c, 0, nrslices[k] - 1)) + Sum(2*Sum(0.5*Sum(X[nsg[k, c + 1, edge2]]*X[nsg[k, c, edge1]]*validtransfer[sg[k, c, edge1], sg[k, c + 1, edge2], k], (edge2, 0, nredges[k, c + 1] - 1)), (edge1, 0, nredges[k, c] - 1)), (c, 0, nrslices[k] - 2)), (k, 0, nrpassengers - 1))

In [21]:
# only one flight in each time slice
# cost of flights
# enough transfer time
# [capacity constraint]
cost_function = sym.Sum(
    sym.Sum(
        P * (sym.Sum(
            X[nsg[k,c,e]]
            , (e, 0, nredges[k,c]-1)
            ) - 1)**2
        , (c, 0, nrslices[k]-1)
    ) + 2 *
    sym.Sum(
        0.5 *
        sym.Sum(
            X[nsg[k,c,e]] * cost[sg[k,c,e]], (e, 0, nredges[k,c]-1)
    )
        , (c, 0, nrslices[k]-1)
    ) + 

    sym.Sum( 2 *
        sym.Sum( 0.5 *
            sym.Sum(
                (X[nsg[k,c,edge1]] * X[nsg[k,c+1,edge2]] * validtransfer[sg[k,c,edge1], sg[k,c+1,edge2], k])
                
                , (edge2, 0, nredges[k,c+1]-1)
            )
            , (edge1, 0, nredges[k,c]-1)
        )
        , (c, 0, nrslices[k]-2)
    )
    
    
    , (k, 0, nrpassengers-1) 
)
cost_function

Sum(Sum(P*(Sum(X[nsg[k, c, e]], (e, 0, nredges[k, c] - 1)) - 1)**2, (c, 0, nrslices[k] - 1)) + 2*Sum(0.5*Sum(X[nsg[k, c, e]]*cost[sg[k, c, e]], (e, 0, nredges[k, c] - 1)), (c, 0, nrslices[k] - 1)) + Sum(2*Sum(0.5*Sum(X[nsg[k, c + 1, edge2]]*X[nsg[k, c, edge1]]*validtransfer[sg[k, c, edge1], sg[k, c + 1, edge2], k], (edge2, 0, nredges[k, c + 1] - 1)), (edge1, 0, nredges[k, c] - 1)), (c, 0, nrslices[k] - 2)), (k, 0, nrpassengers - 1))

cost_function = sym.Sum(
    sym.Sum(
        P * (sym.Sum(
            X[nsg[k,c,e]]
            , (e, 0, nredges[k,c]-1)
            ) - 1)**2
        , (c, 0, nrslices[k]-1)
    ) + 2 *
    sym.Sum(
        0.5 *
        sym.Sum(
            X[nsg[k,c,e]] * flightduration[sg[k,c,e]], (e, 0, nredges[k,c]-1)
    )
        , (c, 0, nrslices[k]-1)
    ) 
    + 

    sym.Sum( 2 *
        sym.Sum( 0.5 *
            sym.Sum(
                (X[nsg[k,c,edge1]] * X[nsg[k,c+1,edge2]] * validtransfer[sg[k,c,edge1], sg[k,c+1,edge2], k])
                + transfertime[sg[k,c,edge1], sg[k,c+1,edge2]]

                
                , (edge2, 0, nredges[k,c+1]-1)
            )
            , (edge1, 0, nredges[k,c]-1)
        )
        , (c, 0, nrslices[k]-2)
    )
    
    
    , (k, 0, nrpassengers-1) 
)
cost_function

In [22]:
nr_qubits

15

In [23]:
single_values_dict = {
    P: penalty,
    nrpassengers: len(passengers), 
    nrflights: len(flights)
    }

nsg_dict = {
    nsg[k,c,e]: test_nsg[k][c][e] 
    for k in range(len(passengers)) 
    for c in range(len(test_nsg[k])) 
    for e in range(len(test_nsg[k][c]))
}

sg_dict = {
    sg[k,c,e]: test_sg[k][c][e] 
    for k in range(len(passengers)) 
    for c in range(len(test_sg[k])) 
    for e in range(len(test_sg[k][c]))
}

nr_slices_dict = {
    nrslices[k]: len(test_sg[k]) 
    for k in range(len(passengers))
}

nr_edges_dict = {
    nredges[k,c]: len(test_sg[k][c]) 
    for k in range(len(passengers)) 
    for c in range(len(test_sg[k]))
}

cost_dict = {
    cost[edge]: flight_cost(edge)
    for k in range(len(passengers)) 
    for c in range(len(test_sg[k])) 
    for edge in test_sg[k][c]
}

valid_transfer_dict = {
    validtransfer[edge1, edge2, k]: transfer_check(edge1, edge2, k) 
    for k in range(len(passengers)) 
    for c in range(len(test_sg[k]) - 1) 
    for edge1 in test_sg[k][c] 
    for edge2 in test_sg[k][c+1]
}

#transfer_time_dict = {
#    transfertime[f1, f2]: transfer_time(f1, f2) 
#    for f1 in flights
#    for f2 in flights
#}

#flight_duration_dict = {
#    flightduration[f]: flights[f][3] - flights[f][2]
#    for f in flights
#}

flight_list_dict = {
    flightlist[f]: flight_list[f]
    for f in range(len(flights))
}

#capacity_dict = {
#    capacity[flight]: capacity_polynomial(flight, len(passengers)) 
#    for flight in flights 
#}

In [24]:
test_sg

[[['A11', 'A12'], ['A13', 'Z1'], ['Z1']],
 [['A11', 'A12'], ['A13', 'Z1'], ['Z1']],
 [['A11', 'A12'], ['A13', 'Z1'], ['Z1']]]

In [25]:
#capacity_dict

In [26]:
cost_dict

{cost[A11]: 150, cost[A12]: 20, cost[A13]: 70, cost[Z1]: 0}

for i in range(len(flights)):
    print(i)

for f in flights:
    print(f)

test_sg

valid_transfer_dict

In [27]:
start = time.time()
cost_poly = sym.Poly(cost_function
                     .subs(single_values_dict)
                     .doit()
                     .subs(nr_slices_dict)
                     .doit()
                     .subs(nr_edges_dict)
                     .doit()
                     .subs(sg_dict)
                     .subs(nsg_dict)
                     .doit()
                     .subs(valid_transfer_dict)
                     #.subs(transfer_time_dict)
                     .subs(cost_dict)
                     #.subs(flight_duration_dict)

                     .subs(flight_list_dict)
                     .doit()
                     #.subs(capacity_dict)
                     .subs(single_values_dict)
                     ,
                     [X[i] for i in range(nr_qubits)])
end = time.time()
print(end-start)
cost_poly

16.865605115890503


Poly(5000*X[0]**2 + 10000*X[0]*X[1] + 5000*X[0]*X[2] - 9850*X[0] + 5000*X[1]**2 + 5000*X[1]*X[2] + 5000*X[1]*X[3] - 9980*X[1] + 5000*X[2]**2 + 10000*X[2]*X[3] - 9930*X[2] + 5000*X[3]**2 - 10000*X[3] + 5000*X[4]**2 - 10000*X[4] + 5000*X[5]**2 + 10000*X[5]*X[6] + 5000*X[5]*X[7] - 9850*X[5] + 5000*X[6]**2 + 5000*X[6]*X[7] + 5000*X[6]*X[8] - 9980*X[6] + 5000*X[7]**2 + 10000*X[7]*X[8] - 9930*X[7] + 5000*X[8]**2 - 10000*X[8] + 5000*X[9]**2 - 10000*X[9] + 5000*X[10]**2 + 10000*X[10]*X[11] + 5000*X[10]*X[12] - 9850*X[10] + 5000*X[11]**2 + 5000*X[11]*X[12] + 5000*X[11]*X[13] - 9980*X[11] + 5000*X[12]**2 + 10000*X[12]*X[13] - 9930*X[12] + 5000*X[13]**2 - 10000*X[13] + 5000*X[14]**2 - 10000*X[14] + 45000, X[0], X[1], X[2], X[3], X[4], X[5], X[6], X[7], X[8], X[9], X[10], X[11], X[12], X[13], X[14], domain='ZZ')

In [28]:
import qiskit
from qiskit.algorithms import QAOA

from qiskit_optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer, CplexOptimizer
from qiskit.utils import QuantumInstance
from qiskit_optimization.problems import QuadraticProgram

# generate qiskit's cost function
qiskit_cost_function = QuadraticProgram()

# define qiskit variables 
for i in range(nr_qubits):#nrqubits in range here
    qiskit_cost_function.binary_var('X_' + str(i))

# specify qiskit cost function
qiskit_cost_function.minimize(
    linear = [int(cost_poly.coeff_monomial(X[i]**1)) for i in range(nr_qubits)], #nrqubits in range here
    quadratic = {
        ('X_'+str(i), 'X_'+str(j)): cost_poly.coeff_monomial(X[i]**1 * X[j]**1)
        for i in range(nr_qubits) #nrqubits in range here
        for j in range(i,nr_qubits) #nrqubits in range here
    }
    )
#for i in range(nr_vehicles):
#    fix_nodes(vehicles[i][0], vehicles[i][1], i, test_sg[i], test_nsg[i], qiskit_cost_function)

print(qiskit_cost_function.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 9850 X_0 - 9980 X_1 - 9930 X_2 - 10000 X_3 - 10000 X_4 - 9850 X_5
      - 9980 X_6 - 9930 X_7 - 10000 X_8 - 10000 X_9 - 9850 X_10 - 9980 X_11
      - 9930 X_12 - 10000 X_13 - 10000 X_14 + [ 10000 X_0^2 + 20000 X_0*X_1
      + 10000 X_0*X_2 + 10000 X_1^2 + 10000 X_1*X_2 + 10000 X_1*X_3
      + 10000 X_2^2 + 20000 X_2*X_3 + 10000 X_3^2 + 10000 X_4^2 + 10000 X_5^2
      + 20000 X_5*X_6 + 10000 X_5*X_7 + 10000 X_6^2 + 10000 X_6*X_7
      + 10000 X_6*X_8 + 10000 X_7^2 + 20000 X_7*X_8 + 10000 X_8^2 + 10000 X_9^2
      + 10000 X_10^2 + 20000 X_10*X_11 + 10000 X_10*X_12 + 10000 X_11^2
      + 10000 X_11*X_12 + 10000 X_11*X_13 + 10000 X_12^2 + 20000 X_12*X_13
      + 10000 X_13^2 + 10000 X_14^2 ]/2
Subject To

Bounds
 0 <= X_0 <= 1
 0 <= X_1 <= 1
 0 <= X_2 <= 1
 0 <= X_3 <= 1
 0 <= X_4 <= 1
 0 <= X_5 <= 1
 0 <= X_6 <= 1
 0 <= X_7 <= 1
 0 <= X_8 <= 1
 0 <= X_9 <= 1
 0 <= X_10 <= 1
 0 <= X_11 <=

quadratic = {
        ('X_'+str(i), 'X_'+str(j)): cost_poly.coeff_monomial(X[i]**1 * X[j]**1)
        for i in range(nr_qubits) #nrqubits in range here
        for j in range(i,nr_qubits) #nrqubits in range here
    }
quadratic

In [29]:
test_sg

[[['A11', 'A12'], ['A13', 'Z1'], ['Z1']],
 [['A11', 'A12'], ['A13', 'Z1'], ['Z1']],
 [['A11', 'A12'], ['A13', 'Z1'], ['Z1']]]

In [33]:
def get_qubit_indices(flight):
    qubit_indices = []

    for k in range(len(passengers)):
        for c in range(len(test_sg[k])):
            for e in range(len(test_sg[k][c])):
                if test_sg[k][c][e] == flight:
                    qubit_indices.append(test_nsg[k][c][e])

    return qubit_indices

In [34]:
def add_capacity_constraints(cost_func, flights, nrpassengers):
    for flight in flights:
        if flight[0] != 'Z':

            capacity = flights[flight].capacity
            upper_bound = nrpassengers if nrpassengers < capacity else capacity
            
            qubit_indices = get_qubit_indices(flight)

            if upper_bound > len(qubit_indices):
                upper_bound = len(qubit_indices)

            linear_coefficients = {} # specifies which qubits and which linear coefficient for each qubit. the coefficients will always be 1, since we just add them

            print(qubit_indices)

            for q in qubit_indices:
                qubit_name = 'X_' + str(q)
                linear_coefficients.update({q: 1})

            #print(linear_coefficients)
            cost_func.linear_constraint(linear=linear_coefficients, sense='<=', rhs=upper_bound, name=('capacity_constraint_' + flight))

In [35]:
add_capacity_constraints(qiskit_cost_function, flights, len(passengers))

[0, 5, 10]
[1, 6, 11]
[2, 7, 12]
[]
[]


In [36]:
#qiskit_cost_function.linear_constraint(linear={'X_0': 1, 'X_5': 1}, sense='<=', rhs=2, name='capacity7')

In [37]:
print(qiskit_cost_function.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 9850 X_0 - 9980 X_1 - 9930 X_2 - 10000 X_3 - 10000 X_4 - 9850 X_5
      - 9980 X_6 - 9930 X_7 - 10000 X_8 - 10000 X_9 - 9850 X_10 - 9980 X_11
      - 9930 X_12 - 10000 X_13 - 10000 X_14 + [ 10000 X_0^2 + 20000 X_0*X_1
      + 10000 X_0*X_2 + 10000 X_1^2 + 10000 X_1*X_2 + 10000 X_1*X_3
      + 10000 X_2^2 + 20000 X_2*X_3 + 10000 X_3^2 + 10000 X_4^2 + 10000 X_5^2
      + 20000 X_5*X_6 + 10000 X_5*X_7 + 10000 X_6^2 + 10000 X_6*X_7
      + 10000 X_6*X_8 + 10000 X_7^2 + 20000 X_7*X_8 + 10000 X_8^2 + 10000 X_9^2
      + 10000 X_10^2 + 20000 X_10*X_11 + 10000 X_10*X_12 + 10000 X_11^2
      + 10000 X_11*X_12 + 10000 X_11*X_13 + 10000 X_12^2 + 20000 X_12*X_13
      + 10000 X_13^2 + 10000 X_14^2 ]/2
Subject To
 capacity_constraint_A11: X_0 + X_5 + X_10 <= 3
 capacity_constraint_A12: X_1 + X_6 + X_11 <= 3
 capacity_constraint_A13: X_2 + X_7 + X_12 <= 3

Bounds
 0 <= X_0 <= 1
 0 <= X_1 <= 1
 0 <=

In [38]:
# execute QAOA on local simulator
qaoa = QAOA(quantum_instance =
             QuantumInstance(backend=qiskit.Aer.get_backend('qasm_simulator'), skip_qobj_validation=False))
optimizer_qaoa = MinimumEigenOptimizer(qaoa)

result_qaoa = optimizer_qaoa.solve(qiskit_cost_function)



results = []

for i in range(1):
    result_qaoa = optimizer_qaoa.solve(qiskit_cost_function)
    results.append(result_qaoa)

In [39]:
results[0]

optimal function value: -39550.0
optimal value: [1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1.]
status: SUCCESS

In [40]:
optimizer = CplexOptimizer() if CplexOptimizer.is_cplex_installed() else None

results_classic = []

for i in range(2):
    result = optimizer.solve(qiskit_cost_function)
    results_classic.append(result)

In [41]:
results_classic

[optimal function value: -44550.0
 optimal value: [1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1.]
 status: SUCCESS,
 optimal function value: -44550.0
 optimal value: [1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1.]
 status: SUCCESS]

In [42]:
test_sg

[[['A11', 'A12'], ['A13', 'Z1'], ['Z1']],
 [['A11', 'A12'], ['A13', 'Z1'], ['Z1']],
 [['A11', 'A12'], ['A13', 'Z1'], ['Z1']]]

In [43]:
passengers

[1, 1, 1]

In [44]:
nr_qubits

15

In [45]:
qubit_results = []
for result in results:
    qubit_string = str(result.x).replace(' ', '').replace('.', '').replace('-', '')[1:-1]
    qubit_results.append(qubit_string)
qubit_results

['100101001110011']

In [46]:
qubit_results[0][0:8]

'10010100'

In [47]:
def interpret_results(result, offset):
    for i in range(len(result)):
        return
        # qubit index is i + offset
        # find the nsg entry equal to qubit index
        # then get the sg entry
        # and return the flight information


In [48]:
#qi = get_qubit_indices('A12')
capacity_polynomial('A12', 4)

[X[11] + X[1] + X[6], X[15] + X[16], X[16], 0]


P*(X[16] - 1)*X[16] + P*(X[15] + X[16] - 2)*X[15] + P*(X[11] + X[1] + X[6] - 3)*(X[11] - X[15] - X[16] + X[1] + X[6])

In [49]:
test_sg

[[['A11', 'A12'], ['A13', 'Z1'], ['Z1']],
 [['A11', 'A12'], ['A13', 'Z1'], ['Z1']],
 [['A11', 'A12'], ['A13', 'Z1'], ['Z1']]]

sort out all flights like in the other preprocessing, plus all flights with capacity 0. obvious, but not trivial i think

number of sums is not always equal to capacity + 1; if the number of passengers is lower then we can instead use that as an upper bound

wait... also if the number of flights is lower. because they can only at most be 1 * flights

In [50]:
minuends = [X[0]] 
polynomial = 0

for i in range(1, 4):
    minuends.append(0)
    for j in range(i, 4):
        minuends[i] += X[number_qubits + j]
minuends.append(0)
    
for i in range(len(minuends) - 1):
    polynomial += ((minuends[i] - (4 - i)) * (minuends[i] - minuends[i + 1]))


minuends
polynomial

NameError: name 'number_qubits' is not defined